In [23]:
# get_data.py
import os
import pandas as pd
from dotenv import load_dotenv
from entsoe import EntsoePandasClient
import plotly.graph_objects as go

load_dotenv()
entsoe_api_key = os.getenv("ENTSOE_API_KEY")

start = pd.Timestamp("20220601", tz="Europe/Brussels")
end = pd.Timestamp("20231201", tz="Europe/Brussels")
country_code = "UK"
output_file_name = "entose_train_data_load.json"


client = EntsoePandasClient(api_key=entsoe_api_key)
df = client.query_load(country_code, start=start, end=end)
# df = client.query_day_ahead_prices(country_code, start=start, end=end)
display(df)
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df["Actual Load"],
#         mode="lines",
#         name="Energy Prices",
#         line=dict(color="blue"),
#     )
# )
# fig.update_layout(
#     title="Energy Prices in London (Past 3 Months)",
#     xaxis_title="Datetime",
#     yaxis_title="Price (Euro/MWh)",
# )
# fig.show()


with open(output_file_name, "w") as file:
    file.write(df.to_json())

,Actual Load
2022-05-31 23:00:00+01:00,800.0
2022-05-31 23:30:00+01:00,750.0
2022-06-01 00:00:00+01:00,696.0
2022-06-01 00:30:00+01:00,655.0
2022-06-01 01:00:00+01:00,630.0
...,...
2023-11-14 11:30:00+00:00,1064.0
2023-11-14 12:00:00+00:00,1053.0
2023-11-14 12:30:00+00:00,1054.0
2023-11-30 11:00:00+00:00,1199.0


pip uninstall tensorflow
pip install tensorflow-cpu


In [21]:
# save_model.py
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Load data
data_filename = "entose_train_data_load.json"
df = pd.read_json(data_filename)
# display(df)

# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_load = scaler.fit_transform(df.values.reshape(-1, 1))

# Reshape for LSTM, y is the next half an hour load
X = []
y = []
for i in range(1, len(scaled_load)):
    X.append(scaled_load[i - 1 : i, 0])
    y.append(scaled_load[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Train
model = Sequential()
model.add(LSTM(50, activation="tanh", input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20


2023-12-17 18:33:39.605275: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-12-17 18:33:39.938471: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-17 18:33:39.938554: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:110] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2023-12-17 18:33:40.040112: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-17 18:33:40.040225: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-17 18:33:40.040256: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-17

InternalError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_4 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykerne